In [1]:
import pandas as pd
import requests
from datetime import datetime
import folium
from IPython.display import IFrame
from flask import Flask, render_template

In [2]:
country_geo = 'world-countries.json'
re = requests.get("https://corona.lmao.ninja/v2/countries?yesterday&sort")
re = re.json()
re = pd.DataFrame.from_dict(re)
country_info = re['countryInfo']
country_info = pd.json_normalize(country_info)
df = pd.concat([re, country_info], axis=1,sort=False)
df = df.drop('countryInfo', axis=1)
pd.options.display.float_format = '{:, .2f}'.format

In [3]:
def top_ten_countries(n=10):
    by_country = df.groupby('country').sum()[['cases']]
    top_ten = by_country.nlargest(n, 'cases')[['cases']]
    top_ten['cases'] = top_ten['cases'].apply(lambda x: "{:,}".format(x))
    return top_ten

d = top_ten_countries()
print(d)

                  cases
country                
USA           5,612,027
Brazil        3,363,235
India         2,703,517
Russia          927,745
South Africa    589,886
Peru            541,493
Mexico          525,733
Colombia        476,660
Chile           387,502
Spain           382,142


In [4]:
def convert_time_stamp(x):
    t = datetime.fromtimestamp(x/1000.0)
    s = t.strftime('%Y-%m-%d %H:%M:%S')
    return s[:-3]
df['updated'] = df['updated'].apply(convert_time_stamp)

In [5]:
df['recovered'] = df['recovered'].apply(lambda x: "{:,}".format(x))
df['cases'] = df['cases'].apply(lambda x: "{:,}".format(x))
df['deaths'] = df['deaths'].apply(lambda x: "{:,}".format(x))

t = df[df['country'] == 'Angola']
new_dict = {}
my_list = ['country', 'cases']
for key, value in t.iteritems():
    if key in my_list:
        print(key +': '+ str(value.values).strip("[]''") +'\n')

country: Angola

cases: 1,935



In [6]:
m = folium.Map(
    location=[100, 0],
    zoom_start=1.5,
)

In [7]:
folium.Choropleth (
    geo_data= country_geo,
    data=df,
    columns=['lat', 'long'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Covid Analysis'
).add_to(m)

In [8]:
def circle_maker(x):
    folium.Circle(location=[x[0],x[1]],
                 radius=10,
                 color="red",
                 popup='{}\n Cases: {}\n Deaths: {}\n Recovered: {}'.format(x[2], x[3], x[4], x[5])).add_to(m)
df[['lat','long','country','cases','deaths', 'recovered']].apply(lambda x:circle_maker(x),axis=1)
html_map = m._repr_html_()

In [9]:
IFrame(src=m, width=450, height=450)
folium.Popup(src=m, max_width=1000)

In [10]:
app = Flask(__name__)

@app.route('/')
def home():
    return render_template("home.html", cmap=html_map)

@app.route('/about')
def about():
    return render_template("about.html")

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use